In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [3]:
dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(dataset_name)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [5]:
dataset['train']['id'][:10]

['train_0',
 'train_1',
 'train_2',
 'train_3',
 'train_4',
 'train_5',
 'train_6',
 'train_7',
 'train_8',
 'train_9']

In [6]:
print(dataset['train']['dialogue'][0])

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.


In [7]:
print(dataset['train']['summary'][0])

Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll give some information about their classes and medications to help Mr. Smith quit smoking.


In [8]:
print(dataset['train']['topic'][0])

get a check-up


In [9]:
checkpoint = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [13]:
sentence = "What time is it, Tom?"
sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
    sentence_encoded['input_ids'][0],
    skip_special_tokens=True
)


print('ENCODED SENTENCE:')
print(sentence_encoded['input_ids'])
print("\nDECODED SENTENCE:")
print(sentence_decoded)

ENCODED SENTENCE:
tensor([[ 363,   97,   19,   34,    6, 3059,   58,    1]])

DECODED SENTENCE:
What time is it, Tom?


In [36]:
for k, v in sentence_encoded.items():
    print(k, " : ",  v)

input_ids  :  tensor([[ 363,   97,   19,   34,    6, 3059,   58,    1]])
attention_mask  :  tensor([[1, 1, 1, 1, 1, 1, 1, 1]])


In [40]:
tokenizer.decode(
    model.generate(
        sentence_encoded['input_ids'], 
        max_new_tokens=50
    )[0].tolist(), 
    skip_special_tokens=True
)

'7:00 PM'

## Summarizing Dialogue without Prompt Engineering

In [19]:
example_indices = [40, 200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    dialogue = dataset['train'][index]['dialogue']
    summary = dataset['train'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens = 50
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Exmaple', i + 1)
    print(f"INPUT PROMPT:\n{dialogue}")
    print(dash_line)
    print(f"BASELINE HUMAN SUMMARY:\n{summary}")
    print(dash_line)
    print(f"MODEL GENERATION:\n{output}\n")

---------------------------------------------------------------------------------------------------
Exmaple 1
INPUT PROMPT:
#Person1#: I just bought a new dress. What do you think of it?
#Person2#: You look really great in it. So are you going to a job interview or a party?
#Person1#: No, I was invited to give a talk in my school.
#Person2#: So how much did you pay for it?
#Person1#: I pay just $70 for it. I saved $30.
#Person2#: That's really a bargain.
#Person1#: You're right. Well, what did you do while I was out shopping?
#Person2#: I watched TV for a while and then I did some reading. It wasn't a very interesting book so I just read a few pages. Then I took a shower.
#Person1#: I thought you said you were going to see Mike.
#Person2#: I'll go and visit him at his home tomorrow. He'll return home tomorrow morning.
#Person1#: I'm glad he can finally returned home after that accident.
---------------------------------------------------------------------------------------------------


## Zero Shot Inference

In [20]:
for i, index in enumerate(example_indices):
    dialogue = dataset['train'][index]['dialogue']
    summary = dataset['train'][index]['summary']

    prompt = f"""
Summarize the following conversation:

{dialogue}

Summary:
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Exmaple', i + 1)
    print(f"INPUT PROMPT:\n{dialogue}")
    print(dash_line)
    print(f"BASELINE HUMAN SUMMARY:\n{summary}")
    print(dash_line)
    print(f"MODEL GENERATION:\n{output}\n")

---------------------------------------------------------------------------------------------------
Exmaple 1
INPUT PROMPT:
#Person1#: I just bought a new dress. What do you think of it?
#Person2#: You look really great in it. So are you going to a job interview or a party?
#Person1#: No, I was invited to give a talk in my school.
#Person2#: So how much did you pay for it?
#Person1#: I pay just $70 for it. I saved $30.
#Person2#: That's really a bargain.
#Person1#: You're right. Well, what did you do while I was out shopping?
#Person2#: I watched TV for a while and then I did some reading. It wasn't a very interesting book so I just read a few pages. Then I took a shower.
#Person1#: I thought you said you were going to see Mike.
#Person2#: I'll go and visit him at his home tomorrow. He'll return home tomorrow morning.
#Person1#: I'm glad he can finally returned home after that accident.
---------------------------------------------------------------------------------------------------


## Zero Shot Inference with Prompt Template from FLAN-T5

In [41]:
for i, index in enumerate(example_indices):
    dialogue = dataset['train'][index]['dialogue']
    summary = dataset['train'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Exmaple', i + 1)
    print(f"INPUT PROMPT:\n{dialogue}")
    print(dash_line)
    print(f"BASELINE HUMAN SUMMARY:\n{summary}")
    print(dash_line)
    print(f"MODEL GENERATION:\n{output}\n")
    

---------------------------------------------------------------------------------------------------
Exmaple 1
INPUT PROMPT:
#Person1#: I just bought a new dress. What do you think of it?
#Person2#: You look really great in it. So are you going to a job interview or a party?
#Person1#: No, I was invited to give a talk in my school.
#Person2#: So how much did you pay for it?
#Person1#: I pay just $70 for it. I saved $30.
#Person2#: That's really a bargain.
#Person1#: You're right. Well, what did you do while I was out shopping?
#Person2#: I watched TV for a while and then I did some reading. It wasn't a very interesting book so I just read a few pages. Then I took a shower.
#Person1#: I thought you said you were going to see Mike.
#Person2#: I'll go and visit him at his home tomorrow. He'll return home tomorrow morning.
#Person1#: I'm glad he can finally returned home after that accident.
---------------------------------------------------------------------------------------------------


## One Shot Inference

In [42]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['train'][index]['dialogue']
        summary = dataset['train'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}



"""
    dialogue = dataset['train'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
    
    return prompt


In [43]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

In [45]:
print(one_shot_prompt)


Dialogue:

#Person1#: I just bought a new dress. What do you think of it?
#Person2#: You look really great in it. So are you going to a job interview or a party?
#Person1#: No, I was invited to give a talk in my school.
#Person2#: So how much did you pay for it?
#Person1#: I pay just $70 for it. I saved $30.
#Person2#: That's really a bargain.
#Person1#: You're right. Well, what did you do while I was out shopping?
#Person2#: I watched TV for a while and then I did some reading. It wasn't a very interesting book so I just read a few pages. Then I took a shower.
#Person1#: I thought you said you were going to see Mike.
#Person2#: I'll go and visit him at his home tomorrow. He'll return home tomorrow morning.
#Person1#: I'm glad he can finally returned home after that accident.

What was going on?
While #Person1# made a bargain to buy a new dress, #Person2# watched TV, read a boring book, and took a shower at home.




Dialogue:

#Person1#: What do you want to know about me?
#Person2#: 

In [46]:
summary = dataset['train'][example_index_to_summarize]['summary']
inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person2# asks #Person1# several questions, like academic records, the highest marks, scholarships, club activities, and skilled sports.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
Person1#: I am a student at college.


## Few Shot Learning

In [47]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: I just bought a new dress. What do you think of it?
#Person2#: You look really great in it. So are you going to a job interview or a party?
#Person1#: No, I was invited to give a talk in my school.
#Person2#: So how much did you pay for it?
#Person1#: I pay just $70 for it. I saved $30.
#Person2#: That's really a bargain.
#Person1#: You're right. Well, what did you do while I was out shopping?
#Person2#: I watched TV for a while and then I did some reading. It wasn't a very interesting book so I just read a few pages. Then I took a shower.
#Person1#: I thought you said you were going to see Mike.
#Person2#: I'll go and visit him at his home tomorrow. He'll return home tomorrow morning.
#Person1#: I'm glad he can finally returned home after that accident.

What was going on?
While #Person1# made a bargain to buy a new dress, #Person2# watched TV, read a boring book, and took a shower at home.




Dialogue:

#Person1#: Hey, man. What's up?
#Person2#: Ah, first of a

In [63]:
summary = dataset['train'][example_index_to_summarize]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50,
        do_sample=True,
        temperature=2.0
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person2# asks #Person1# several questions, like academic records, the highest marks, scholarships, club activities, and skilled sports.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
In any booze competition that you don’t work hard at it really works to improve it while at it you are with your school classmates. So the he thinks you ought


## Generative Configuration Parameters for Inference

In [64]:
generative_config = GenerationConfig(max_new_tokens=50)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        generative_config=generative_config
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

ValueError: The following `model_kwargs` are not used by the model: ['generative_config'] (note: typos in the generate arguments will also show up in this list)